In this project you will use and modify the module you developed previously (File Encryption). You'll also use the [OS package](https://docs.python.org/3/library/os.html) as well as [JSON](https://docs.python.org/3/library/json.html) package.

Step 1:

Modify your File Encryption to include the policy of Encrypt-then-[MAC](https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/) for every encryption.

(C, IV, tag)= **MyencryptMAC**(message, EncKey, HMACKey)

(C, IV, tag, Enckey, HMACKey, ext)= **MyfileEncryptMAC** (filepath)

(RSACipher, C, IV, tag, ext)= **MyRSAEncrypt**(filepath, RSA_Publickey_filepath)

Where in the above, RSACipher includes RSA encryption of m=EncKey+ HMACKey (concatenated). Make sure you implement the reverse of the above (MACVerification-then-Decrypt). You can use SHA256 in your [HMAC](https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/).

Save this class as FileEncryptMAC.

Step 2:

Next, you will a script that looks for a pair of RSA Public and private key (using a CONSTANT file path; PEM format). If the files do not exist (use OS package) then generate the RSA public and private key (2048 bits length) using the same constant file path.

Step 3:

You can use the OS package to retrieve the current working directory. Then you can get a list of all files in this directory. For each file, encrypt them using **MyRSAEncrypt** from your new FileEncryptMAC module. Do this in a loop for all files (make sure you do not encrypt the RSA Private Key file). For every file that is encrypted, store the encrypted file as a JSON file. The attributes you have for each file are 'RSACipher', 'C', 'IV', 'tag' and 'ext'. The values are from MyRSAEncrypt method. Once the JSON fire is written (use json.dump() with file.write() methods) into a JSON file then you can remove the plaintext file (use os.remove() method).

Note: For now, you can skip encrypting files within directories in the working directories (i.e., recursive execution).

Note: DO NOT test your script on any valuable file. It will be your responsibility if you lose any important data to you.

Step 4:

Using [Pyinstaller](http://www.pyinstaller.org/) or [Py2exe](http://www.py2exe.org/) create an executable file from your step 3.

Do NOT run the executable file on important folders. Only test on a designated python working directory. You are responsible if you lose any important file.

## (C, IV, tag)= MyencryptMAC(message, EncKey, HMACKey)

In [126]:
import os
import base64
import cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, serialization, hashes, asymmetric as asymm, hashes, hmac
from cryptography.hazmat.primitives.asymmetric import rsa

# encryption method AES-CBC-256, Encrypt-then-MAC(SHA256)
def MyencryptMAC(message, EncKey, HMACKey):
    
    #step 1: Encrypt
    #key length check
    if len(EncKey)<32:
        return "Error: This place is full of land mines, dragons, and dinosaurs with laser guns. Increase you key length to upgrade your armor."
    
    try:
        message = message.encode()
    except:
        pass
    
    iv = os.urandom(16)   #generate an iv

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message) + padder.finalize()
    message = padded_data
        
    #calling the default AES CBC mode
    cipher = Cipher(algorithms.AES(EncKey), modes.CBC(iv), backend=default_backend())
    #creating an encryptor object
    encryptor = cipher.encryptor()
    #generating cipher text
    ct = encryptor.update(message) + encryptor.finalize()
    
    #step 2: HMAC
    # create a tag
    tag = hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
    tag.update(ct)
    tag = tag.finalize()
        
    return(ct, iv, tag) 


# MACVerification-then-Decrypt
def MydecryptMAC(ct, iv, tag, EncKey, HMACKey):

    #Step 1: Verify
    h= hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
    h.update(ct)
    h = h.finalize()
    
    if h == tag:
        print("Tag verified")
        cipher = Cipher(algorithms.AES(EncKey), modes.CBC(iv), backend=default_backend())
        #creating a decryptor object
        decryptor = cipher.decryptor()
        pt = decryptor.update(ct) + decryptor.finalize()

        try:
            unpadder = padding.PKCS7(128).unpadder()
            pt = unpadder.update(pt) + unpadder.finalize()
            print("MydecryptMAC Complete.")
            return pt
        except:
            return pt

    else:
        return("Invalid tag")


# execution code    
EncKey = os.urandom(32)
HMACKey = os.urandom(32)

m = "a secret message"
C, IV, tag = MyencryptMAC(m, EncKey, HMACKey)
print("Encrypted Message with a tag:\n")
print(C,"\n", IV, "\n", tag)
print("\nDecrypted Message:")
MydecryptMAC(C, IV, tag, EncKey, HMACKey)

Encrypted Message with a tag:

b'\xb2\x00\x0b\xa3\x9d9?!A\xbe\xebE\xea\xb6\xd7\xad\xd7\xb5\x15J\xab=^\xc7\xe2\x9c\xb5NE\x1f\xa6\x94' 
 b'w\xcb&\x8cW\xf1\xf0\x93D\x91\xdd\rX\xdcf\xf3' 
 b'\x1b\xc2\xf0+\x0f\xc0\xf5\x1fXa0\xcbpb\x88\xfd;g(=\x03\xf5\xaa^\xcb\xd4\xc6\x15\r\x9a\xb4\x88'

Decrypted Message:
Tag verified
MydecryptMAC Complete.


b'a secret message'

## (C, IV, tag, Enckey, HMACKey, ext)= **MyfileEncryptMAC** (filepath)


In [130]:
# file encryption algorithm
def MyfileEncryptMAC(filepath):
    key = os.urandom(32)
    h = os.urandom(32)
    # Read the entire file as a single byte string
    with open(filepath, 'rb') as f:
        data = f.read()

    result = MyencryptMAC(data, key, h)
    ext = os.path.splitext(file_path)[1]
    result += (key, h, ext)
    
    image_result = open("encrypted_image.png", 'wb') # create a writable image and write the decoding result
    image_result.write(result[0])
    print("Encryption+Tag finished.\n")
    return result

# file dencryption algorithm
def MyfileDecryptMAC(enc_filepath, iv, tag, key, h, ext):
    
    with open(enc_filepath, 'rb') as f:
        data = f.read()
    
    file_name = "decrypted_image" + ext
    plaintext = MydecryptMAC(data, iv, tag, key, h)
    image_result = open(file_name, 'wb') # create a writable image and write the decoding result
    image_result.write(plaintext)
    print("MyfileDecryptMAC Complete.")


# ----------------------
#Execution code
file_path = os.path.abspath("hello.png")
C, IV, tag, Enckey, HMACKey, ext = MyfileEncryptMAC(file_path)
# print(C,"\n",IV,"\n",tag,"\n",Enckey,"\n",HMACKey,"\n",ext)
MyfileDecryptMAC("encrypted_image.png", IV, tag, Enckey, HMACKey, ext)

Encryption+Tag finished.

Tag verified
MydecryptMAC Complete.
MyfileDecryptMAC Complete.


## (RSACipher, C, IV, tag, ext)= **MyRSAEncrypt**(filepath, RSA_Publickey_filepath)

In [133]:
#create public/private key pair
def create_pem_key_pair():
    # create key object
    backend = default_backend()
    key = rsa.generate_private_key(backend=backend, public_exponent=65537,key_size=2048)
    
    # private key
    private_key = key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
            )
    with open("private.pem", 'wb') as private_pem:
        private_pem.write(private_key)
        private_pem.close()
    
    #public key
    public_key = key.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
    with open("public.pem", 'wb') as public_pem:
        public_pem.write(public_key)
        public_pem.close()
    
    
#RSA encryption method
def MyRSAEncryptMAC(file_path, RSA_Publickey_filepath):
    #encrypting an image file
    C, IV, tag, Enckey, HMACKey, ext = MyfileEncryptMAC(file_path)
    
    with open(RSA_Publickey_filepath, "rb") as p_key:
        public_key = serialization.load_pem_public_key(p_key.read(),backend=default_backend())
        
    #obtain RSACipher
    RSACipher = public_key.encrypt(Enckey+ "break".encode() +HMACKey, asymm.padding.OAEP(
                                           mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),
                                           algorithm=hashes.SHA256(),
                                           label=None ))

    return RSACipher, C, IV, tag, ext



#RSA decryption method
def MyRSADecryptMAC(RSACipher, enc_file_path, iv, tag, ext, RSA_Privatekey_filepath):

    with open(RSA_Privatekey_filepath, "rb") as Enckey:
        private_key = serialization.load_pem_private_key(Enckey.read(),password=None, backend=default_backend())
    
    key = private_key.decrypt(
        RSACipher,
        asymm.padding.OAEP(mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    
    # split the Enc_key and HMAC_key
    key_list = key.split("break".encode())
    Enckey, HMACKey = key_list[0], key_list[1]
    MyfileDecryptMAC(enc_file_path,         iv, tag, Enckey, HMACKey, ext)


#execution code
create_pem_key_pair() 
RSACipher, C, IV, tag, ext = MyRSAEncryptMAC("hello.png", "public.pem")

# enc_file = os.path.abspath("encrypted_image.png")
MyRSADecryptMAC(RSACipher, "encrypted_image.png", IV, tag, ext, "private.pem")

Encryption+Tag finished.

Tag verified
MydecryptMAC Complete.
MyfileDecryptMAC Complete.
